In [3]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import time
import pickle
import datetime

In [4]:
company_name = pd.read_csv('company_names.csv', engine = 'python')
company_name.loc[company_name['name'].str.contains('아모레')]

,name
74,아모레퍼시픽
75,아모레퍼시픽우
76,아모레G


In [321]:
options = webdriver.ChromeOptions()
options.add_argument('headless') 
options.add_argument('window-size = 1920x1980')

driver = webdriver.Chrome('chromedriver.exe')
driver.get('https://news.nate.com')

driver.find_element_by_id('input_search').send_keys('아모레퍼시픽')

driver.find_element_by_id('acBtn').click()

iframe = driver.find_element_by_id('contentsWraper').find_element_by_tag_name('iframe')
driver.switch_to_frame(iframe)

driver.find_element_by_css_selector('dl.option-list > dd.opt02 > span.sel02').click() # 검색영역 '제목' 선택

# 기간 설정
driver.find_element_by_name('ps1').clear()
driver.find_element_by_name('ps1').send_keys('20100806')
driver.find_element_by_name('ps2').clear()
driver.find_element_by_name('ps2').send_keys('20180912')


driver.find_element_by_class_name('type-search-btn').click()
while True:
    try:
        if driver.find_element_by_css_selector('div.none-result'):
            driver.refresh()
    except Exception as e:
        print(e)
        break
time.sleep(1)

test = 400
url_list = []
while True:
    for i in range(test):
        page_num = (i + 1)
        
        try:
            titles = driver.find_elements_by_css_selector('dt.text-inline')
            
            if not titles:
                iframe = driver.find_element_by_id('contentsWraper').find_element_by_tag_name('iframe')
                driver.switch_to_frame(iframe)
                titles = driver.find_elements_by_css_selector('dt.text-inline')
            for j in range(len(titles)):
                #print(titles[j].find_element_by_tag_name('a').get_attribute('href'))
                url_list.append(titles[j].find_element_by_tag_name('a').get_attribute('href'))
        except Exception as e:
            print(e)
            print('이번 페이지는 크롤링 못 했다. (%d페이지)' %page_num)
            
        if i % 100 == 0:
            print('진행상황 ({}/400)'.format(i))
            pd.Series(url_list).to_csv('news_data/news{}.csv'.format(i), index = False)
            url_list = []
        if i == (test-1):
            break
        driver.find_element_by_css_selector('.section.news.last > div.paging').find_element_by_link_text('다음').click()
        time.sleep(1)
    break

Message: no such element: Unable to locate element: {"method":"css selector","selector":"div.none-result"}
  (Session info: chrome=70.0.3538.77)
  (Driver info: chromedriver=2.41.578737 (49da6702b16031c40d63e5618de03a32ff6c197e),platform=Windows NT 10.0.17134 x86_64)

진행상황 (0/400)
Message: no such element: Unable to locate element: {"method":"id","selector":"contentsWraper"}
  (Session info: chrome=70.0.3538.77)
  (Driver info: chromedriver=2.41.578737 (49da6702b16031c40d63e5618de03a32ff6c197e),platform=Windows NT 10.0.17134 x86_64)

이번 페이지는 크롤링 못 했다. (2페이지)


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":".section.news.last > div.paging"}
  (Session info: chrome=70.0.3538.77)
  (Driver info: chromedriver=2.41.578737 (49da6702b16031c40d63e5618de03a32ff6c197e),platform=Windows NT 10.0.17134 x86_64)


In [203]:
# url로 접근
options = webdriver.ChromeOptions()
options.add_argument('headless') 
options.add_argument('window-size = 1920x1980')

driver = webdriver.Chrome('chromedriver.exe')
driver.get('https://news.nate.com')

driver.find_element_by_id('input_search').send_keys('아모레퍼시픽')

driver.find_element_by_id('acBtn').click()

iframe = driver.find_element_by_id('contentsWraper').find_element_by_tag_name('iframe')
driver.switch_to_frame(iframe)

#driver.find_element_by_css_selector('dl.option-list > dd.opt02 > span.sel02').click() # 검색영역 '제목' 선택

# 기간 설정
driver.find_element_by_name('ps1').clear()
driver.find_element_by_name('ps1').send_keys('20100806')
driver.find_element_by_name('ps2').clear()
driver.find_element_by_name('ps2').send_keys('20180912')
time.sleep(2)


driver.find_element_by_class_name('type-search-btn').click()
justin = 0
while True:
    try:
        if driver.find_element_by_css_selector('div.none-result'):
            driver.refresh()
    except Exception as e:
        print(e)
        break
time.sleep(1)

lastdate = datetime.datetime.strptime('20180912', '%Y%m%d')
url_list = []
p = 20
while True:
    print('jjjjjjjjjjjjj : ', justin)
    print('While문 진입')
    try:
        #if driver.find_element_by_id('contentsWraper').find_element_by_tag_name('iframe'):
        iframe = driver.find_element_by_id('contentsWraper').find_element_by_tag_name('iframe')
        driver.switch_to_frame(iframe)
        page_url = driver.find_element_by_css_selector('div.paging > span > a').get_attribute('href')
        page_url = page_url[:-2]
        for i in range(p):
            try:
                URL = page_url + str(1 + (i * 10))
                driver.get(URL)
                time.sleep(1)
                titles = driver.find_elements_by_css_selector('dt.text-inline')
                
                if p > 10:
                    print(i, '   이건데 왜 이거 실행함?')
                    dates = [day.text[-16:-6].replace('.', '') for day in driver.find_elements_by_css_selector('ul.search-list dl dd.text-inline span')]
                    dtime = [datetime.datetime.strptime(d, '%Y%m%d') for d in dates]
                    if lastdate > min(dtime):
                        lastdate = min(dtime)
                    print(lastdate + '   *' * 100)                
                
                for j in range(len(titles)):
                    url_list.append(titles[j].find_element_by_tag_name('a').get_attribute('href').replace('https://news.nate.com/view/', ''))
                if i == 99:
                    print('진행상황 ({}/99)'.format(i + justin*100))
                    pd.Series(url_list).to_csv('news_data/news{}.csv'.format(i + justin * 100), index = False)
                    url_list = []
            except Exception as e:
                print(e)
                print('이번 페이지는 크롤링 못 했다. (%d페이지)' %(i+1))
                URL = page_url + str(1 + (i * 10))
                driver.get(URL)
                time.sleep(1)
                titles = driver.find_elements_by_css_selector('dt.text-inline')
                
                if p > 10:
                    print(i, '   이건데 왜 이거 실행함?')                    
                    dates = [day.text[-16:-6].replace('.', '') for day in driver.find_elements_by_css_selector('ul.search-list dl dd.text-inline span')]
                    dtime = [datetime.datetime.strptime(d, '%Y%m%d') for d in dates]
                    if lastdate > min(dtime):
                        lastdate = min(dtime)
                    print(lastdate + '   *' * 100)
                    
                for j in range(len(titles)):
                    url_list.append(titles[j].find_element_by_tag_name('a').get_attribute('href').replace('https://news.nate.com/view/', ''))
                if i == 99:
                    print('진행상황 ({}/99)'.format(i + justin * 100))
                    pd.Series(url_list).to_csv('news_data/news{}.csv'.format(i + justin * 100), index = False)
                    url_list = []
                
    except Exception as e:
        print(e)
        print('No iframe')
        #iframe = driver.find_element_by_id('contentsWraper').find_element_by_tag_name('iframe')
        #driver.switch_to_frame(iframe)
        page_url = driver.find_element_by_css_selector('div.paging > span > a').get_attribute('href')
        page_url = page_url[:-2]
        for i in range(p):
            try:
                print(i,' *' * 100)
                URL = page_url + str(1 + (i * 10))
                driver.get(URL)
                time.sleep(1)
                titles = driver.find_elements_by_css_selector('dt.text-inline')
                
                if p > 10:                
                    dates = [day.text[-16:-6].replace('.', '') for day in driver.find_elements_by_css_selector('ul.search-list dl dd.text-inline span')]
                    dtime = [datetime.datetime.strptime(d, '%Y%m%d') for d in dates]
                    if lastdate > min(dtime):
                        lastdate = min(dtime)
                    print(lastdate, '   *' * 100)                
                
                for j in range(len(titles)):
                    url_list.append(titles[j].find_element_by_tag_name('a').get_attribute('href').replace('https://news.nate.com/view/', ''))
                if  i == 99:
                    print('진행상황 ({}/99)'.format(i + justin * 100))
                    pd.Series(url_list).to_csv('news_data/news{}.csv'.format(i + justin*100), index = False)
                    url_list = []
                    
            except Exception as e:
                print(i,' * ' * 100)                
                print(e)
                print('이번 페이지는 크롤링 못 했다. (%d페이지)' %(i+1))
                URL = page_url + str(1 + (i * 10))
                driver.get(URL)
                time.sleep(1)
                titles = driver.find_elements_by_css_selector('dt.text-inline')
                
                if p > 10:                
                    dates = [day.text[-16:-6].replace('.', '') for day in driver.find_elements_by_css_selector('ul.search-list dl dd.text-inline span')]
                    dtime = [datetime.datetime.strptime(d, '%Y%m%d') for d in dates]
                    if lastdate > min(dtime):
                        lastdate = min(dtime)
                    print(lastdate, '   *' * 100)                
                
                for j in range(len(titles)):
                    url_list.append(titles[j].find_element_by_tag_name('a').get_attribute('href').replace('https://news.nate.com/view/', ''))
                if  i == 99:
                    print('진행상황 ({}/99)'.format(i + justin * 100))
                    pd.Series(url_list).to_csv('news_data/news{}.csv'.format(i + justin * 100), index = False)
                    url_list = []
        
    # 마지막 페이지의 마지막 글의 날짜
    #date_list = driver.find_elements_by_css_selector('ul.search-list dl dd.text-inline span')
    #date = date_list[9].text[-16:-6].replace('.', '')
    
    dateinput = lastdate.strftime('%Y%m%d')
    driver.find_element_by_name('ps1').clear()
    driver.find_element_by_name('ps1').send_keys('20100806')
    driver.find_element_by_name('ps2').clear()
    driver.find_element_by_name('ps2').send_keys(dateinput)
    time.sleep(2)
    print(dateinput)
    #driver.find_element_by_css_selector('dl.option-list > dd.opt02 > span.sel02').click() # 검색영역 '제목' 선택
    driver.find_element_by_class_name('type-search-btn').click()
    print('endendendendenden : ', justin)
    justin = justin + 1
    if justin == 123:
        break

Message: no such element: Unable to locate element: {"method":"css selector","selector":"div.none-result"}
  (Session info: chrome=70.0.3538.77)
  (Driver info: chromedriver=2.41.578737 (49da6702b16031c40d63e5618de03a32ff6c197e),platform=Windows NT 10.0.17134 x86_64)

jjjjjjjjjjjjj :  0
While문 진입
Message: no such element: Unable to locate element: {"method":"id","selector":"contentsWraper"}
  (Session info: chrome=70.0.3538.77)
  (Driver info: chromedriver=2.41.578737 (49da6702b16031c40d63e5618de03a32ff6c197e),platform=Windows NT 10.0.17134 x86_64)

No iframe
0  * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * *
0  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  

11  * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * *
2018-07-16 00:00:00    *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *
12  * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * *
2018-07-16 00:00:00    *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   

3  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  * 
min() arg is an empty sequence
이번 페이지는 크롤링 못 했다. (4페이지)
2018-03-05 00:00:00    *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *
4  * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * *
2018-02-01 00:00:

16  * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * *
2018-02-01 00:00:00    *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *
17  * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * *
2018-02-01 00:00:00    *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   

9  * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * *
2017-11-24 00:00:00    *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *
10  * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * *
2017-11-24 00:00:00    *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *

2  * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * *
2017-11-21 00:00:00    *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *
3  * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * *
2017-11-20 00:00:00    *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   * 

2017-11-14 00:00:00    *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *
16  * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * *
2017-11-14 00:00:00    *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   *   

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"div.paging > span > a"}
  (Session info: chrome=70.0.3538.77)
  (Driver info: chromedriver=2.41.578737 (49da6702b16031c40d63e5618de03a32ff6c197e),platform=Windows NT 10.0.17134 x86_64)


In [ ]:
lastdate = datetime.datetime.strptime('20180912', '%Y%m%d')

                if p > 10:                
                    dates = [day.text[-16:-6].replace('.', '') for day in driver.find_elements_by_css_selector('ul.search-list dl dd.text-inline span')]
                    dtime = [datetime.datetime.strptime(d, '%Y%m%d') for d in dates]
                    if lastdate > min(dtime):
                        lastdate = min(dtime)
                    print(lastdate + '   *' * 100)
                    
dateinput = lastdate.strftime('%Y%m%d')

In [196]:
URL

'https://news.search.nate.com/iframe/search.html?q=%EC%95%84%EB%AA%A8%EB%A0%88%ED%8D%BC%EC%8B%9C%ED%94%BD&f3=2&ps=3&ps1=20100806&ps2=20180912&o=P&e=1'

In [194]:
page_url

'https://news.search.nate.com/iframe/search.html?q=%EC%95%84%EB%AA%A8%EB%A0%88%ED%8D%BC%EC%8B%9C%ED%94%BD&e='

In [190]:
lastdate = datetime.datetime.strptime('20180912', '%Y%m%d')
lastdate.strftime('%Y%m%d')

'20180912'

In [189]:
date

datetime.date

In [182]:
datetime.datetime.strptime('20180912', '%Y%m%d') > datetime.datetime.strptime('20180712', '%Y%m%d')

True

In [ ]:
iframe = driver.find_element_by_id('contentsWraper').find_element_by_tag_name('iframe')
driver.switch_to_frame(iframe)

In [96]:
titles[1].

<selenium.webdriver.remote.webelement.WebElement (session="1165ca3e7bcf97ac636003424f336c86", element="0.0968357145162082-2")>

# 새로 배운 사실
1. try except은 혁명이다.
2. 크롤링을 못 하도록 수법을 걸어놨는데, iframe페이지로 들어가니 상관 없어짐

In [114]:
df = pd.read_csv('news_data/news99.csv', header = None)
df.columns = ['URL']
df.head()
df.size

480

In [166]:
from datetime import date
a = date(2006, 1, 6)
b = date(2006, 1, 7)
a<b

True

In [180]:
datetime.datetime.strptime('20160101', '%Y%m%d')

datetime.datetime(2016, 1, 1, 0, 0)

In [187]:
import datetime
a = '20060108'
mydate = datetime.datetime.strptime(a, '%Y%m%d')
print(mydate)

datetime.datetime.now().strftime('%Y%m%d')

2006-01-08 00:00:00


datetime.datetime(2018, 11, 13, 21, 40, 32, 325660)

In [177]:
d = ['20180108', '20170820', '20160914', '20160913']
mydate = [datetime.datetime.strptime(dd, '%Y%m%d') for dd in d]
mydate

[datetime.datetime(2018, 1, 8, 0, 0),
 datetime.datetime(2017, 8, 20, 0, 0),
 datetime.datetime(2016, 9, 14, 0, 0),
 datetime.datetime(2016, 9, 13, 0, 0)]

In [178]:
min(mydate)

datetime.datetime(2016, 9, 13, 0, 0)

In [170]:
date_list[0].text

WebDriverException: Message: chrome not reachable
  (Session info: chrome=70.0.3538.77)
  (Driver info: chromedriver=2.41.578737 (49da6702b16031c40d63e5618de03a32ff6c197e),platform=Windows NT 10.0.17134 x86_64)


In [119]:
df['URL'].iloc[0]
driver = webdriver.Chrome('chromedriver.exe')

In [120]:
driver.get(url + df['URL'].iloc[0])

In [130]:
import glob
import pandas as pd

In [109]:
url = 'https://news.nate.com/view/'

In [153]:
path = 'news_data'
url = 'https://news.nate.com/view/'

all_files = glob.glob(path + '/*.csv')
df_list = [pd.read_csv(file, encoding = 'utf-8', header = None) for file in all_files]
df = pd.concat(df_list, sort = False)
df.columns = ['URL']

In [154]:
df.size

123590

In [157]:
from datetime import date
date(20180701)

TypeError: Required argument 'month' (pos 2) not found

In [150]:
df.drop_duplicates('URL', keep = 'first', inplace = True)
url_list = set(df['URL'].values)

In [152]:
len(url_list)

9301

In [149]:
driver = webdriver.Chrome('chromedriver.exe')

title = []
content = []
title = []
reporter = []
date = []

for u in url_list:
    driver.get(url + u)
    title.append(driver.find_element_by_css_selector('#articleView h3').text)

WebDriverException: Message: chrome not reachable
  (Session info: chrome=70.0.3538.77)
  (Driver info: chromedriver=2.41.578737 (49da6702b16031c40d63e5618de03a32ff6c197e),platform=Windows NT 10.0.17134 x86_64)


In [123]:
title = []
content = []
title = []
reporter = []
date = []
for i in range(10):
    driver.get(url + df['URL'].iloc[i])
    titledriver.find_element_by_css_selector('#articleView h3').text)

[ST포토] 하정우 등장에 아모레퍼시픽 직원들도 들썩
아모레퍼시픽, '글로벌 바이오 콘퍼런스'서 맞춤형 화장품 선봬
[한경로보뉴스] '아모레퍼시픽' 5% 이상 상승, 외국계 증권사 창구의 거래비중 25% 수준
아모레퍼시픽, 2018년 글로벌 바이오 콘퍼런스 참가
'인재의 힘이 기업 성장동력' 부러운 아모레퍼시픽 '워라밸'
아모레퍼시픽, 글로벌 바이오 콘퍼런스 참가
내 피부에 꼭 맞는 립컬러는?…아모레퍼시픽 맞춤형 화장품 공개
아모레퍼시픽, 2018년 글로벌 바이오 콘퍼런스 참가
[게시판] 아모레퍼시픽, 글로벌 바이오 콘퍼런스 참가
아모레퍼시픽, '글로벌 바이오 콘퍼런스'서 맞춤형 서비스 소개


In [128]:
[(1,2,3,4), (5,6,7,8)]

TypeError: list indices must be integers or slices, not tuple

In [ ]:
current_window = driver.current_window_handle
driver.switch_to.window(driver.window_handles[1])

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
title = soup.select('h3.articleSubecjt')[0].text.strip()
date = soup.select('span.firstDate > em')[0].text.strip()

In [149]:
content = soup.find('div', {'id' : 'realArtcContents'})
content.text.strip()

'강원랜드 본사 ⓒ News1(정선=뉴스1) 박하림 기자 = 강원랜드(대표 문태곤)가 본사가 위치한 정선읍 고한에 두 번째 직장어린이집을 짓는다.강원랜드는 정부의 일·가정 양립정책 적극 이행 및 임직원의 복지향상을 위해 총 39억7400만 원을 투자해 강원랜드 행정동(강원도 정선군 고한로 55) 옆 잔디광장에 연면적 842.95㎡(약 255평) 규모의 (가칭)하이원 정선어린이집을 건립하기로 했다.이번에 조성되는 어린이집은 2012년 태백시에 개원한 하이원 태백어린이집에 이은 두 번째 직장 보육시설로 75명의 아동을 돌볼 수 있는 5실의 유아보육실을 비롯해 교재교구실, 다목적 아동 전용식당, 강당, 실외 놀이터 등의 공간으로 조성될 예정이다.강원랜드는 2021년 개원을 목표로 올해 안에 설계 발주에 들어갈 예정이다.강원랜드는 11일 오전 10시 강원도 정선 본사와 서울사무소간 화상회의시스템을 통해 열린 제161차 이사회에서 이 같은 내용을 담은 제2 직장어린이집 신축공사(안)을 심의해 원안 의결했다.뉴스1 관련뉴스☞ 김병준 "박근혜 재판은 정치 재판…홍준표는 평당원"☞ 또래 소녀가 성매수자 물색…채팅앱 청소년 성매매 적발☞ 이재명 "국토보유세 신설해 불로소득 국민에 돌려줘야"☞ 사찰서 60대 승려, 50대女 살해 후 목숨 끊어☞ 합의성관계 후 9차례 강간당했다 무고한 女rimrock@news1.kr▶ 뉴스1 100% 무료 만화 서비스 오픈!![ⓒ 뉴스1코리아(news1.kr), 무단 전재 및 재배포 금지]'

In [145]:
content

[<p><br/></p>,
 <p style="font-size: 12px !important; color:#777;font-family: '돋움',dotum;">강원랜드 본사 ⓒ News1</p>,
 <p><br/></p>,
 <p><br/>(정선=뉴스1) 박하림 기자 = 강원랜드(대표 문태곤)가 본사가 위치한 정선읍 고한에 두 번째 직장어린이집을 짓는다.<br/><br/>강원랜드는 정부의 일·가정 양립정책 적극 이행 및 임직원의 복지향상을 위해 총 39억7400만 원을 투자해 강원랜드 행정동(강원도 정선군 고한로 55) 옆 잔디광장에 연면적 842.95㎡(약 255평) 규모의 (가칭)하이원 정선어린이집을 건립하기로 했다.<br/><br/>이번에 조성되는 어린이집은 2012년 태백시에 개원한 하이원 태백어린이집에 이은 두 번째 직장 보육시설로 75명의 아동을 돌볼 수 있는 5실의 유아보육실을 비롯해 교재교구실, 다목적 아동 전용식당, 강당, 실외 놀이터 등의 공간으로 조성될 예정이다.<br/><br/>강원랜드는 2021년 개원을 목표로 올해 안에 설계 발주에 들어갈 예정이다.<br/><br/>강원랜드는 11일 오전 10시 강원도 정선 본사와 서울사무소간 화상회의시스템을 통해 열린 제161차 이사회에서 이 같은 내용을 담은 제2 직장어린이집 신축공사(안)을 심의해 원안 의결했다.<br/><br/></p>]